
# 09 — MCP With Agents (Tool-Using Agents Skeleton)

This notebook shows a simple pattern for **agents using tools via an MCP-style registry**:

- A `ToolRegistry` that mimics MCP tool discovery/calls
- A few example tools (calculator, fake web search, notes)
- An `AgentWithTools` that decides which tool to call
- A small LangGraph workflow wiring state + agent + tools


## 0. Setup

In [ ]:

!pip install langchain langchain-openai langchain-community langgraph --quiet

from typing import Dict, Any, Callable, TypedDict, List
from dataclasses import dataclass
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(model="gpt-4o-mini")


## 1. MCP-Style Tool Registry Skeleton

In [ ]:

class ToolRegistry:
    """Simple MCP-like tool registry.

    In real MCP:
    - Tools are exposed by servers
    - Clients discover available tools and schemas
    - Calls are structured requests/responses
    """
    def __init__(self):
        self.tools: Dict[str, Callable[..., Any]] = {}

    def register(self, name: str, fn: Callable[..., Any]):
        self.tools[name] = fn

    def call(self, name: str, **kwargs) -> Any:
        if name not in self.tools:
            raise ValueError(f"Tool '{name}' not found")
        return self.tools[name](**kwargs)

tool_registry = ToolRegistry()
tool_registry


## 2. Example Tools (Calculator, Web Search, Notes)

In [ ]:

# These are simple Python functions, but you can imagine them as MCP tools.

def tool_calculator(expression: str) -> str:
    try:
        # Very basic eval; for demo only
        result = eval(expression, {"__builtins__": {}})
        return f"Result of {expression} = {result}"
    except Exception as e:
        return f"Error evaluating expression: {e}"

def tool_fake_web_search(query: str) -> str:
    # Stand-in for a real HTTP-based search MCP tool
    return f"[Fake search results for '{query}': (1) RAG docs, (2) Agents docs, (3) MCP protocol spec]"

_notes_store: List[str] = []

def tool_notes_add(text: str) -> str:
    _notes_store.append(text)
    return f"Note added. Total notes: {_notes_store.__len__()}"

def tool_notes_list() -> str:
    if not _notes_store:
        return "[No notes stored]"
    return "Notes:\n- " + "\n- ".join(_notes_store)

# Register tools in registry
tool_registry.register("calculator", tool_calculator)
tool_registry.register("web_search", tool_fake_web_search)
tool_registry.register("notes_add", tool_notes_add)
tool_registry.register("notes_list", lambda: tool_notes_list())

# Quick test
print(tool_registry.call("calculator", expression="2 + 3 * 4"))
print(tool_registry.call("web_search", query="RAG with agents"))
print(tool_registry.call("notes_add", text="Study MCP protocol"))
print(tool_registry.call("notes_list"))


## 3. AgentWithTools (decides which tool to call)

In [ ]:

@dataclass
class ToolDecision:
    tool_name: str
    args: Dict[str, Any]

class AgentWithTools:
    """Agent that:
    - Parses user request
    - Decides which tool (if any) to call
    - Calls the tool via registry
    - Returns a combined answer
    """
    def __init__(self, llm: ChatOpenAI, registry: ToolRegistry):
        self.llm = llm
        self.registry = registry

    def decide_tool(self, question: str) -> ToolDecision | None:
        """Very simple keyword-based routing for demo purposes."""
        q = question.lower()
        if any(x in q for x in ["calculate", "calc", "math", "sum", "add", "minus", "multiply"]):
            expr = question.replace("calculate", "").replace("calc", "")
            return ToolDecision(tool_name="calculator", args={"expression": expr.strip()})
        if any(x in q for x in ["search", "google", "lookup"]):
            return ToolDecision(tool_name="web_search", args={"query": question})
        if "add note" in q or "note:" in q:
            # naive extraction
            note_text = question.split("note:", 1)[-1].strip() if "note:" in q else question
            return ToolDecision(tool_name="notes_add", args={"text": note_text})
        if "list notes" in q or "show notes" in q:
            return ToolDecision(tool_name="notes_list", args={})
        return None

    def run(self, question: str) -> str:
        decision = self.decide_tool(question)
        if decision:
            tool_result = self.registry.call(decision.tool_name, **decision.args)
            prompt = f"""You are an assistant working with tools.

User question:
{question}

Tool called: {decision.tool_name}
Tool result:
{tool_result}

Explain the result to the user in plain language.
"""
            return self.llm.invoke(prompt).content
        else:
            # Fallback: pure LLM
            return self.llm.invoke(question).content

agent = AgentWithTools(llm, tool_registry)
print(agent.run("Calculate 5 * (3 + 2)"))
print(agent.run("Search about RAG agents MCP"))
print(agent.run("Add note: Finish Agents Universe notebooks"))
print(agent.run("List notes"))


## 4. LangGraph Workflow (State + Agent + Tool Calls)

In [ ]:

class MCPState(TypedDict):
    question: str
    answer: str

def mcp_agent_node(state: MCPState) -> MCPState:
    result = agent.run(state["question"])
    state["answer"] = result
    return state

graph = StateGraph(MCPState)
graph.add_node("mcp_agent", mcp_agent_node)
graph.set_entry_point("mcp_agent")
graph.add_edge("mcp_agent", END)

workflow = graph.compile()

print(workflow.invoke({"question": "Calculate 10 * 10 + 5"}))
print(workflow.invoke({"question": "List notes"}))



## End of Notebook 09 — MCP With Agents

You now have:

- A MCP-style `ToolRegistry`
- Example tools (calculator, search, notes)
- An `AgentWithTools` that routes questions to tools
- A simple LangGraph workflow that wraps the agent

Next step (optional): connect to real MCP servers and tools.
